In [101]:
import torch
import pandas as pd
import torch.utils
from torch import nn

In [102]:
#Loading house prices training and test dataset
hptrain = pd.read_csv("/Users/bartoszgawin/Desktop/ML/Practise/PyTorch_Basic/Datasets/hptrain.csv",index_col=0)
hptest = pd.read_csv("Datasets/hptest.csv",index_col=0)

In [103]:
#Transforming data into tensor data format from pandas' dataframe
X_ols_train = torch.tensor(hptrain[['LotArea', 'OverallQual', 'OverallCond', 'MasVnrArea', 'TotalBsmtSF']].values,requires_grad=True).detach()
print(X_ols_train.is_leaf)
Y_ols_train = torch.tensor(hptrain[['SalePrice']].values).detach()


X_ols_train = X_ols_train.float().detach()
Y_ols_train = Y_ols_train.float().detach()



print(X_ols_train.is_leaf)


#Splitting data into iterable batches
olstensor = torch.utils.data.TensorDataset(X_ols_train,Y_ols_train)
olsdataloader = torch.utils.data.DataLoader(olstensor, batch_size=20)

True
True


In [104]:
#Specifying the paramenters of our model
olsnet = nn.Sequential(nn.Linear(X_ols_train.size()[1], Y_ols_train.size()[1]))
olsnet[0].weight.data.normal_(0, 10)
olsnet[0].bias.data.normal_(0, 10)
olsloss = nn.MSELoss()
olstrainer = torch.optim.SGD(olsnet.parameters(),lr=0.03)

In [114]:
olsnet[0].weight.data.normal_(0, 0.1)
olsnet[0].bias.data.normal_(0, 0.1)

num_epochs = 5
for epoch in range(num_epochs):
    print('-------------------')
    print(f'{epoch+1}th epoch')
    print('===================')
    olstrainer.zero_grad()
    X, Y = next(iter(olsdataloader))
    X.requires_grad = True
    X.retain_grad()
    Y.requires_grad = True
    Y.retain_grad()
    print('It is a leaf: ',X.is_leaf)
    l = olsloss(olsnet(X),Y).detach()
    l.requires_grad = True
    l.retain_grad()
    print(f'Loss:{l}')
    l.backward()
    olstrainer.step()
    print(f'Gradient:{l.grad}')
    print('Coefficients: ',olsnet[0].weight.data)





-------------------
1th epoch
It is a leaf:  True
Loss:37426749440.0
Gradient:1.0
Coefficients:  tensor([[-0.0437, -0.0248,  0.0174,  0.0571,  0.0472]])
-------------------
2th epoch
It is a leaf:  True
Loss:37426749440.0
Gradient:1.0
Coefficients:  tensor([[-0.0437, -0.0248,  0.0174,  0.0571,  0.0472]])
-------------------
3th epoch
It is a leaf:  True
Loss:37426749440.0
Gradient:1.0
Coefficients:  tensor([[-0.0437, -0.0248,  0.0174,  0.0571,  0.0472]])
-------------------
4th epoch
It is a leaf:  True
Loss:37426749440.0
Gradient:1.0
Coefficients:  tensor([[-0.0437, -0.0248,  0.0174,  0.0571,  0.0472]])
-------------------
5th epoch
It is a leaf:  True
Loss:37426749440.0
Gradient:1.0
Coefficients:  tensor([[-0.0437, -0.0248,  0.0174,  0.0571,  0.0472]])
